# Replication

This function is this formula:
$$
\pi_{j t}^{i}(p_{j t})=\frac{\exp(x_{j t}\beta_{i}-\alpha_{i}p_{j t})}{1+\exp(x_{j t}\beta_{i}-\alpha_{i}p_{j t})} = \frac{1}{1-\exp(- x_{j t}\beta_{i} + \alpha_{i}p_{j t})}
$$
 
Note that in the product share function, consumer preferences are negative in value. These numbers and $\beta$ are provided in the code as starting value.

    `bL = -1.05185291` 
    `bB = -0.72189149` 

Because the set of prices $A(t)$, is calculated from the k-means algorithm, we used the original code to produce the prices and use them in our model.

The prices are calculated as:

`prices = [1.93584019, 2.51706816, 3.06912093, 3.68721778, 4.35668806, 5.03306089, 5.8804312 , 6.74935852]` 



$$
\pi_{j t}(p_{j t})= \gamma_{t}\,\pi_{j t}^{B}({\mathcal{P}}_{j t})+\left(1 - \gamma_{t}\right)\pi_{j t}^{L}({\mathcal{P}}_{j t})
$$

After calculating $\pi_{j t}(p_{j t})$, we proceed with demand probability:
$$
\underset{t} {Pr_{t}}(Q_{j t} = q; P_{j t}) = \frac{\left(\mu_{t} \pi_{j t} \right)^{q} \exp \left(-\mu_{t} \pi_{j t}\right)}{q!} q
$$



$$Pr_{t}(\text{Business}) =  \gamma_{t}=\frac{\exp{(\gamma_{0} + \gamma_{1} t+\gamma_{2} t^{2}})}{1+\exp{(\gamma_{0}+ \gamma_{1} t+ \gamma_{2} t^{2})}} = $$

In [4]:
PR_B(γ_0, γ_1,γ_2, T) = 1 ./ (exp.(-γ_0 .- (0:(T-1)) .* γ_1 .- ((0:(T-1)) .^ 2) .* γ_2) .+ 1)

PR_B (generic function with 1 method)

In [1]:
function log_demandQ_tγ(β, bL, bB, γt, μt, q, prices)
    """
        input:
            γt: Consumer type prob at time t 1x1
            μt: mean demand at time t 7x1
            q : seat count 1x1
            β : demand parameters 7x1
            bL: Leisure demand parameters 1x1
            bB: Business demand parameters 1x1

        output: 
            log of demand probability matrix 8x7
    """
    #* Product share:

    # Probability of Leisure booking on time t
    πL_t = (1 .-γt) ./(1 .+ exp.(-β' .- bL .* prices))
    # Probability of Business booking on time t
    πB_t = (γt) ./(1 .+ exp.(-β' .- bB .* prices))

    π_t = πL_t .+ πB_t
    # --------------------------------------------------------------------------

    #* Log of Demand probability:

    # demand calculation for time t, seat count q
    Q_jt = q.*(log.(π_t) .+ log.(μt)') .- (π_t'.*μt)'  .- loggamma(q+1)
    # res = res .|> x -> ifelse((x > 0) & (x < 1e-100), 1e-100, x)
    return Q_jt
end

#* Demand probability:
# for Q == q at time t
demandQ_tγ(β, bL, bB, γt, μt, q, prices) = exp.(log_demandQ_tγ(β, bL, bB, γt, μt, q, prices))

# 
Demand_q_T(β, bL, bB, γ, μ, Δq, Pt) = ([demandQ_tγ(β, bL, bB, γt, μ[t,:], Δq, Pt) for (t,γt) in enumerate(γ)]);

In [ ]:
function matrix_f(β, bL, bB, γ, μ, q̄, Pt)
    Prob_sellout = []
    for Δq in 0:(q̄)
        temp = Demand_q_T(β, bL, bB, γ, μ, Δq, Pt)
        push!(Prob_sellout, temp)
    end
    f = Dict()
    f[0] = [[zeros(8,7) for i in 1:60]]

    for Δq in 1:(q̄)
        f[Δq] = []
        for Pq in 1:(Δq)
            push!(f[Δq], Prob_sellout[Pq])
        end
        push!(f[Δq], [max.((1 .- t), 1e-100) for t in sum(f[Δq])])
        ER0[Δq] = sum([(f[Δq][i+1] .|> x -> Pt*(i) .* x) for i in 0:(Δq)])
    end
return (f, ER0)
end


In [ ]:
function ER(β, bL, bB, γ, μ, q̄, Pt)
    Prob_sellout = []
    for Δq in 0:(q̄)
        temp = Demand_q_T(β, bL, bB, γ, μ, Δq, Pt)
        push!(Prob_sellout, temp)
    end
    f = Dict()
    f[0] = [[zeros(8,7) for i in 1:60]]
    ER0 = Dict()
    ER0[0] = [zeros(8,7) for i in 1:60]
    for Δq in 1:(q̄)
        f[Δq] = []
        for Pq in 1:(Δq)
            push!(f[Δq], Prob_sellout[Pq])
        end
        push!(f[Δq], [max.((1 .- t), 1e-100) for t in sum(f[Δq])])
        ER0[Δq] = sum([(f[Δq][i+1] .|> x -> Pt*(i) .* x) for i in 0:(Δq)])
    end
return (f, ER0)
end


We proceed with expected revenues:
 $$
  R^{e}_{t}(P_{t};c_{t}) = p_{t} \cdot Q^{e}_{t}(P_{t};c_{t})
 $$

In [2]:

gpr_qt(ER, EV, Pₜ, t, σ) = ((ER + EV) ./ σ) .* Pₜ[t, 2:end]

V_T(grp_q, EC, σ) = σ * (log.(sum(exp.(grp_q), dims=1)) .+ EC)

inf_to_zero(x) = ifelse.(x .== -Inf, 0, x);

function dynEst(f, ER , σ, T, Pₜ ,q̄)
    println("works!")
    t = T
    # generating the EV at t = T, where EV(T+1) = 0
    EV = Dict()
    V = Dict()
    CCP = Dict()
    
    EV[t+1] = [zeros(8,7) for i in 1:q̄]
    grp = []

    for q in 1:(q̄)
        push!(grp, gpr_qt(ER[q-1][t], EV[t+1][q] , Pₜ, t, σ))
    end

    grp = grp .|> x-> ifelse.(x .== 0, -Inf, x);
    V[t] = V_T.(grp, EC, σ);
    CCP[t] = grp .|> x -> x .- log.(sum(exp.(x), dims=1));
    replace!.(CCP[t],NaN => 0.0)

    EV[t] = [zeros(8,7) for i in 1:q̄]
    
    for t in (T-1):-1:2
        grp = []
        for q in 1:(q̄)
            push!(grp, gpr_qt(ER[q-1][t], EV[t+1][q] , Pₜ , t, σ))
        end
        grp = grp .|> x-> ifelse.(x .== 0, -Inf, x);
        CCP[t] = grp .|> x -> x .- log.(sum(exp.(x), dims=1));
        replace!.(CCP[t],NaN => 0.0)

        V[t] = V_T.(grp, EC, σ);
        
        EV[t] = []
        for q in 1:(q̄)
            g_q = f[q-1] .|> x -> x[t-1]
            V_q = collect(vcat([zeros(1,7)], V[t][1:(length(g_q)-1)]) )
        
            for q_r in 1:q
                push!(EV[t], g_q[q_r] .* V_q[q_r])
            end
        end
    end
    return CCP
end


dynEst (generic function with 1 method)

In [3]:
# q: seat remaining
# Δq: seat change
current_dir = @__DIR__

# Get the parent directory
parent_dir = dirname(current_dir)

# Change the working directory to the parent directory
cd(parent_dir)

using DataFrames, JuMP
using Statistics, Clustering, HypothesisTests, StatsBase, SpecialFunctions
using Query, FreqTables
using ShiftedArrays
using Dates, Missings, Random
using Pipe: @pipe
import Parquet2, CSV


In [ ]:
function ER_f(β, bL, bB, γ, μ, q̄, Pt)
    Prob_sellout = []
    for Δq in 0:(q̄)
        temp = Demand_q_T(β, bL, bB, γ, μ, Δq, Pt)
        push!(Prob_sellout, temp)
    end
    f = Dict()
    f[0] = [[zeros(8,7) for i in 1:60]]
    ER0 = Dict()
    ER0[0] = [zeros(8,7) for i in 1:60]
    for Δq in 1:(q̄)
        f[Δq] = []
        for Pq in 1:(Δq)
            push!(f[Δq], Prob_sellout[Pq])
        end
        push!(f[Δq], [max.((1 .- t), 1e-100) for t in sum(f[Δq])])
        ER0[Δq] = sum([(f[Δq][i+1] .|> x -> Pt*(i) .* x) for i in 0:(Δq)])
    end
return (f, ER0)
end


In [ ]:

function logLike(X₀, data, T, q̄, Pt)
    # Variables specs:
    """
    T: Max number of days before departure
    t: time ∈ {,1,...,T}
    q̄: maximum seat count
    Δq: change in seats ∈ {0,1,..., q̄}
    Pt: clustered prices
    γ: probability of type Business
    μ[t,:] : Arrival rates (to be estimated)
    bL: Leisure demand parameter (to be estimated)
    bB: Business demand parameter (to be estimated)
    β : demand parameters (to be estimated)
    """

    EC = 0.5772156649 # Euler constant

    X₀ = [ # Initial values
        2.49999999, 2.49999999, 2.49999999, 2.49999999, 2.49999999, 
        2.49999999, 2.49999999, -1.05185291, -0.72189149, -13.39650409, 
        0.27373386, 0.0, 1.91183252, 2.46138227, 1.82139054, 2.35728083, 
        1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.22463165
    ]
    # price at time t
    Pt = [1.93584019, 2.51706816, 3.06912093, 3.68721778, 4.35668806, 5.03306089, 5.8804312 , 6.74935852];

    VAR = X₀

    q̄ = 121

    β = X₀[1:7]
    bL = min(X₀[8], X₀[9])
    bB = max(X₀[8], X₀[9])
    
    γ = PR_B(X₀[10], X₀[11],X₀[12], T) = 1 ./ (exp.(-γ_0 .- (0:(T-1)) .* γ_1 .- ((0:(T-1)) .^ 2) .* γ_2) .+ 1)

    μT = [X₀[13] .* ones(T - 20); X₀[14] .* ones(7); X₀[15] .* ones(7); X₀[16] .* ones(6)]
    μD = [1; X₀[17:22]]
    μ = μT * μD'
    σ = X₀[end];

    f, ER0 = ER_f(β, bL, bB, γ, μ, q̄, Pt)
    
    loss0 = []
    for x in 1:size(data)[1]
        push!(loss0, f[data[x,1]][data[x,2]+1][data[x,3]+1][data[x,4]+1, data[x,5]+1])
    end
    loss0 = sum(log.(loss0))
    
    CCP0 = dynEst(f, ER0, σ, T,Pₜ)
    loss1 = []
    for x in 1:size(data)[1]
        push!(loss1, CCP0[data[x,1]][data[x,2]+1][data[x,3]+1][data[x,4]+1, data[x,5]+1])
    end
    loss1 = sum(loss1)

    return loss0 + loss1
end


In [ ]:
Pₜ = Matrix(Pₜ);

In [ ]:
data = data[data[:,1].<=60,:]

In [ ]:
logLike(X₀, data, 60, q̄, Pt)

## Expected return and transition matrix $f$

In [ ]:
using Optim # for BFGS -> KN_HESSOPT_BFGS
using MKL # Using Julia with Intel's MKL -> KN_BLASOPTION_INTEL
using NLopt

bndsLo = [
        -10, -10, -10, -10, -10, -10, -10, -10, -10,
        -250, -10, -.06, .1, .1, .1, .1,
        .01, .01, .01, .01, .01, .01, .02
    ];
bndsUp = [
    15, 15, 15, 15, 15, 15, 15, 0, 0, 40, 10, .15,
    15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 2
];

X₀ = [ # Initial values
    2.49999999, 2.49999999, 2.49999999, 2.49999999, 2.49999999, 
    2.49999999, 2.49999999, -1.05185291, -0.72189149, -13.39650409, 
    0.27373386, 0.0, 1.91183252, 2.46138227, 1.82139054, 2.35728083, 
    1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.22463165
];
Pt = [1.93584019, 2.51706816, 3.06912093, 3.68721778, 4.35668806, 5.03306089, 5.8804312 , 6.74935852];

df = DataFrame(CSV.File("G:/My Drive/Courses/ECON-L6210 - Structural econometrics/Assignments/Presentation/welfare-airlines-main/Replication/data.csv"));
data = convert.(Int,Matrix(df))[:,2:end];

Pₜ = DataFrame(CSV.File("G:/My Drive/Courses/ECON-L6210 - Structural econometrics/Assignments/Presentation/welfare-airlines-main/Replication/Pt.csv"));

# df = DataFrame(CSV.File("Replication/data.csv"));
# data = convert.(Int,Matrix(df))[:,2:end];

VAR = X₀;
nothing



In [ ]:
n = length(X₀)
model = Model(NLopt.Optimizer)
set_optimizer_attribute(model, "method", BFGS())


@variable(
    model, 
    X[i = 1:n], 
    start = X₀[i], 
    lower_bound = bndsLo[i], 
    upper_bound = bndsUp[i]
    )



@NLobjective(model, )

In [ ]:
# Create model for optimization
# Round df preparation:

df = Parquet2.Dataset("data/efdata_clean.parquet") |> DataFrame

function determine_OD(O,D)
    res = string(min(O,D), "_", max(O,D))
    return res
end

df[!, "route"] = determine_OD.(df[!,"origin"], df[!,"dest"])

df[!, "ttdate" ] = -df.tdate .+ 60

df = sort(df, [:origin, :dest, :ddate, :flightNum, :tdate], rev = true)

cols = ["origin", "dest", "ddate", "flightNum"]

df.difS = combine(
    groupby(df,cols),
    :seats => x -> ShiftedArrays.lead(x) - x; 
    renamecols=false
    ).seats

df.difP = combine(
    groupby(df,cols),
    :fare => x -> ShiftedArrays.lead(x) - x; 
    renamecols=false
    ).fare

df[!, :dd_dow] = dayofweek.(df.ddate).-1

MARKET = "BOS_SEA"
df.route|> unique
df_route = df[df.route .== MARKET, :]
df_route = df_route[.!ismissing.(df_route.difS), :]
df_route.difS = ifelse.(df_route.difS .> 0, 0, df_route.difS)
df_route.difS = abs.(df_route.difS)

df_route = df_route[df_route.seats .> 0,:]
df_route.tdate = maximum(df_route.tdate) .- df_route.tdate

# condition for BOS_MCI
if MARKET == "BOS_MCI"
    println("date filtered for BOS_MCI")
    df_route = df_route[df_route.ddate .> Date("2012-05-17"), :]
end

# FROM PAPER:
    # Next, winsorize the data to remove entries in which a large number of seats disappear
    # This could happen when:
    #  - seat maps get smaller
    #  - seat map errors
    #  - measurement error in processing data
    #  - Delta market has more errors which influences log-like, constrain data more.


mark = ifelse(MARKET == "BOS_MCI", 0.985, 0.995)
df_route = df_route[df_route.difS .< quantile(df_route.difS, mark), :]


numFlights = @pipe df_route[!, ["flightNum", "ddate"]] |> unique |> size |> _[1] 
numDDates = @pipe df_route[!, ["ddate"]] |> unique |> size |> _[1]
numObs = size(df_route)[1]
df_route = df_route[!, ["fare", "tdate", "seats", "difS", "dd_dow"]]
df_route = df_route[.!ismissing.(df_route.fare), :]

k=8


it = 2
while true
    k = it
    fares = copy(collect(skipmissing(df_route.fare))) # correcting data type

    Random.seed!(567)
    kmean_res = kmeans(fares', k, tol=1e-4, init=Vector(1:k)) 
    rank_conversion = Dict(zip(sortperm(kmean_res.centers, dims=2, rev=true), 1:it))
    cents = sort(kmean_res.centers, dims=2, rev=true)
    assignments = get.(Ref(rank_conversion), kmean_res.assignments, missing)


    df_route[!,"fareI"] = assignments
    df_route[!,"fareC"] = cents[assignments]

    cc = cor(df_route.fare, df_route.fareC)^2

    println(it, ":", round(cc, digits = 2))

    it += 1
    if round(cc,digits=2) >= 0.99
        it -= 1
        println("Found it! : ", it)
        break
        
    end

end

prices = @pipe df_route.fareC |> unique |> sort |> collect |> _ ./ 100

T = length(countmap(df_route.tdate))
countmap(df_route.fareI)

data = convert.(Int,Matrix(df_route[!, ["seats", "difS", "tdate", "fareI", "dd_dow"]] ))

Pₜ = df_route[!, ["tdate", "fareI"]] |> unique 
Pₜ = unstack(Pₜ, :tdate, :fareI, 1, combine=sum)
Pₜ = sort(Pₜ, :tdate)
Pₜ = Pₜ[!, (1:it .|> x -> string(x))]
Pₜ = hcat((1:T).-1, (.!ismissing.(Pₜ) .* 1))
Pₜ = Matrix(Pₜ)

q̄ = Int(maximum(df_route.seats)+1)

numP = length(prices)
obs = length(df_route.tdate)
# This block of code creates the core estim data as well as key data summaries that enter LLN

first =  map(x -> x[1],argmin(Pₜ, dims = 1)) .-1
last = T .- map(x -> x[1], argmax(reverse(Pₜ, dims = (1,2)), dims = 1)) .+ 1

EC = 0.5772156649 # Euler constant

X₀ = [ # Initial values
    2.49999999, 2.49999999, 2.49999999, 2.49999999, 2.49999999, 
    2.49999999, 2.49999999, -1.05185291, -0.72189149, -13.39650409, 
    0.27373386, 0.0, 1.91183252, 2.46138227, 1.82139054, 2.35728083, 
    1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.22463165
]

VAR = X₀;